# Marker Genes and Annotation

In [3]:
import scanpy as sc
import pandas as pd

adata = sc.read(r"C:\Users\jtspy\OneDrive\Desktop\PersonalProject\NSCLC_Project\NSCLC-scRNA-Analysis\data\processed\adata_umap_clusters_leiden0.6.h5ad")
